In [48]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Blackbox Attack

The way the blackbox attack works is as follows:
Since we do not know details about the model we are attacking, we train a "substitute model" to approximate the "target model".
This is done using a small sample of training data, labeled by the target model instead of using the ground-truth labels. 
In addition, we augment this training data using "Jacobian-based data augmentation" (TODO)
Finally, we use one of the regular attacks on the substitute model. Supposedly, an attack that works well against the substitute
will also work well against the target. 
"""

In [49]:
import os.path
import matplotlib.pyplot as plt
from IPython.display import Image 
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset

import team36
from team36.mnist.data_loading import MNIST_Loader
from team36.mnist.vgg import VGG
from team36.mnist.cnn import CNN
from team36.attacks.fast_gradient_attack_data_set import FastSignGradientAttackDataSet
from team36.training import train, validate, accuracy, predict, predict_from_loader, do_training, load_or_train

DIR = '.'
DATA_DIR = f'{DIR}/data'

In [ ]:
"""
Step 1
Train the target model (aka the Oracle) on a large portion of the training data, or load a checkpoint
"""

In [2]:
mnist_loader = MNIST_Loader(DIR, DATA_DIR)

C:\Users\bkest\miniconda3\envs\cs7643-final-project-cpu\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
data = torchvision.datasets.MNIST(root=DATA_DIR, train=True, download=True, 
                                          transform=transforms.ToTensor())

In [4]:
oracle = VGG() # set the target model here
oracle_checkpoint = 'mnist-vgg.pth'
EPOCHS = 2
load_or_train(oracle, oracle_checkpoint, epochs=2)
# if os.path.exists(oracle_checkpoint_path): # if trained checkpoint exists, load it
#     state_dict = torch.load(f"{DIR}/checkpoints/{checkpoint}", map_location=torch.device('cpu'))
#     model.load_state_dict(state_dict)
# else: # else, train the model
#     oracle_train_split, oracle_val_split = mnist_loader.train_val_split()
#     print(f"{len(oracle_train_split)} in training set")
#     print(f"{len(oracle_val_split)} in validation set")
#     do_training(oracle, training_split=oracle_train_split, validation_split=oracle_val_split, epochs=EPOCHS)

In [ ]:
"""
Step 2
Train the substitute model on a small portion of the training data, using the oracle's predictions as the labels

oracle pred full mnist
replace targets
train val split (small train)
"""

In [84]:
substitute = CNN()
substitute_checkpoint = 'substitute.pth'

In [70]:
loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=False, num_workers=2)

In [71]:
%%time
oracle_preds = predict_from_loader(oracle, loader)

Wall time: 1min 15s


In [72]:
oracle_preds = oracle_preds.argmax(axis=1)

In [73]:
oracle_preds.shape

torch.Size([60000])

In [79]:
len(oracle_preds[oracle_preds != data.targets]) # check oracle preds mostly matches true labels

0

In [78]:
data.targets = oracle_preds

In [85]:
load_or_train(substitute, substitute_checkpoint, epochs=5)

In [ ]:
"""
Step 3
Generate attack data for the substitute model
"""

In [ ]:
"""
Step 4
Test the target model against the above attack data
"""